In [1]:
import os

In [2]:
pwd

'c:\\Users\\hp\\Desktop\\kidney-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Desktop\\kidney-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [19]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.DataIngestionConfig

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_files(self)->str:
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f'Downloading Data from the {dataset_url} into file {zip_download_dir}')

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f'Downloaded data from {dataset_url} into file{zip_download_dir}')

        except Exception as e:
            raise e
        
    def extract_zip_files(self):
        unzip_path = self.config.unzip_path
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [21]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_zip_files()
except Exception as e:
    raise e

[2023-12-25 21:24:09,261: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-25 21:24:09,263: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-25 21:24:09,264: INFO: common: create directory at artifacts]
[2023-12-25 21:24:09,266: INFO: common: create directory at artifacts/data_ingestion]
[2023-12-25 21:24:09,267: INFO: 2039209928: Downloading Data from the https://drive.google.com/file/d/1yUYxvoOouVZkkkAZ6q9i6aDbTYuqm1de/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1yUYxvoOouVZkkkAZ6q9i6aDbTYuqm1de
From (redirected): https://drive.google.com/uc?/export=download&id=1yUYxvoOouVZkkkAZ6q9i6aDbTYuqm1de&confirm=t&uuid=e696f2f6-9231-4270-8239-fb79058d07e8
To: c:\Users\hp\Desktop\kidney-disease-classification\artifacts\data_ingestion\data.zip
  9%|▉         | 88.1M/940M [00:15<02:31, 5.61MB/s]

: 